# Creating a bioMod file
This file shows how to create a `.bioMod` file, what should be declared, what it means, etc.
Additionnally, we explore how to generate the file programmatically. 
Please note that nothing prevents you from writting the file by hand.

In a nutshell a `.bioMod` file is a description of a rigid body kinematic chain. 
It therefore expect a serie of segments linked by degree of freedom, up to six (up to three translations and three rotations). 
Once loaded, if a degree of freedom is given to the model, then one can interact with it on this degree of freedom.

If one is interested only by the kinematics, then the inertial parameters can be omitted. 
Otherwise, they have to be declared.

The first section of this replicates the model that can be found in `models/SimpleBody.bioMod`

## Make our life easier
First of all, let's make our life easier by defining a class that we will fill for each of the segments and markers.

This section helps automatically write the file. 
If you want to write it by hand, you can skip this section.

In [ ]:
import os 

# Prepare the name of the output file and create a 'models' folder if needed
kinematic_model_file_path = "models/SimpleBody.bioMod"
dynamic_model_file_path = "models/SimpleBodyWithInertia.bioMod"

if not os.path.isdir("models"):
    os.mkdir("models")

In [ ]:
# First of all, let's make our life easier by defining a class that we will fill for each of the segment
# Notice that mass, center_of_mass and inertia_xxyyzz are currently ignored. They will become relevant in 
# dynamics section below
class Segment():
    def __init__(
        self, 
        name, 
        parent_name="", 
        rt="",
        translations="", 
        rotations="",
        mass=0,
        center_of_mass=None,
        inertia_xxyyzz=None,
        mesh: tuple=None,
        markers: tuple=None,
    ):
        self.name = name
        self.parent_name = parent_name
        self.rt = rt
        self.translations = translations
        self.rotations = rotations
        self.mass = mass
        self.center_of_mass=center_of_mass
        self.inertia_xxyyzz=inertia_xxyyzz
        self.mesh = mesh
        self.markers = markers

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"segment {self.name}\n"
        if (self.parent_name):
            out_string += f"\tparent {self.parent_name}\n"
        if (self.rt):
            out_string += f"\tRT {self.rt}\n"
        if (self.translations):
            out_string += f"\ttranslations {self.translations}\n"
        if (self.rotations):
            out_string += f"\trotations {self.rotations}\n"
        if (self.mass):
            out_string += f"\tmass {self.mass}\n"
        if (self.center_of_mass):
            out_string += f"\tcom {self.center_of_mass[0]} {self.center_of_mass[1]} {self.center_of_mass[2]}\n"
        if (self.inertia_xxyyzz):
            out_string += f"\tinertia {self.inertia_xxyyzz[0]} 0 0\n" + \
                          f"\t        0 {self.inertia_xxyyzz[1]} 0\n" + \
                          f"\t        0 0 {self.inertia_xxyyzz[2]}\n"
        if (self.mesh):
            for m in self.mesh:
                out_string += f"\tmesh {m[0]} {m[1]} {m[2]}\n"
        out_string += "endsegment\n"
        
        # Also print the markers attached to the segment
        if (self.markers):
            for marker in self.markers:
                out_string += str(marker)
        return out_string


# Here is an example on how to use this class
my_new_segment = Segment(
    name="my_new_semgent_name", 
    parent_name="the_name_of_the_parent_segment", 
    rt="0 0 0 xyz 0 0 0",  # The transformation between the segment and its parent. The first three values are the three rotations about the given sequence (here xyz) and then the three translations
    translations="yxz",  # The degrees of fredom in translation
    rotations="xyz",  # The degrees of fredom in rotation
    mesh=((0, 0, 0), (0, 0, 0.3))  # The list of mesh point to show to create the stick figure. All the points are link in order
)
print(my_new_segment)  # Later this print should be sent to a file

In [ ]:
# Let's do the same for the markers
class Marker():
    def __init__(
        self, 
        name, 
        parent_name, 
        position,
    ):
        self.name = name
        self.parent_name = parent_name
        self.position = position

    def __str__(self):
        # Define the print function so it automatically format things in the file properly<
        out_string = f"marker {self.name}\n"
        out_string += f"\tparent {self.parent_name}\n"
        out_string += f"\tposition {self.position[0]} {self.position[1]} {self.position[2]}\n"
        out_string += "endmarker\n"
        return out_string

# Here is an example on how to use this class
my_new_marker = Marker(
    name="my_new_marker_name",
    parent_name="the_name_of_the_parent_segment",
    position=(0, 0, 1),
)
print(my_new_marker)  # Later this print should be sent to a file

In [ ]:
# And why not creating a data structure that will collect all the segments and produce the a full
# kinematic chain that can easily be printed
class KinematicChain():
    def __init__(self, segments):
        self.segments = segments
        
    def __str__(self):
        out_string = "version 4\n\n"
        for segment in self.segments:
            out_string += str(segment)
            out_string += "\n\n\n"  # Give some space between segments
        return out_string
    
    def write(self, file_path):
        # Method to write the current KinematicChain to a file
        with open(file_path, "w") as file:
            file.write(str(self))

## Create the `SimpleBody.bioMod` kinematic chain
Now we are ready to produce the kinematic chain.
It is a 2D model made from the mean dimension of the adjusted De Leva model (see the doc folder):
- A TRUNK segment that includes the HEAD (up to the ear) at $72cm$ that moves frontward and upward (translations y and z) and rotations about the medio-lateral axis (rotations x);
- An ARM segment attached to the TRUNK at $53cm$ that includes the HAND at $61cm$. It rotates about the medio-lateral axis (rotations x);
- An THIGH segment attached to the TRUNK. It rotates about the medio-lateral axis (rotations x)
- An SHANK segment attached to the THIGH at $42cm$. It rotates about the medio-lateral axis (rotations x)
- An FOOT segment attached to the SHANK at $43cm$ that includes a FOOT of $25cm$. It rotates about the medio-lateral axis (rotations x)

In [ ]:
# Let's define the segments and the skin markers attached to each segment 
# and collect all the values in the KinematicChain class

# The trunk segment
trunk_marker_pelvis = Marker(
    name="PELVIS", 
    parent_name="TRUNK",
    position=(0, 0, 0),
)
trunk = Segment(
    name="TRUNK",
    translations="yz", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, 0.53)),
    markers=[trunk_marker_pelvis]
)



# The head segment
top_head_marker_head = Marker(
    name="TOP_HEAD",
    parent_name="HEAD",
    position=(0, 0, 0.24),
)
head = Segment(
    name="HEAD",
    parent_name="TRUNK",
    rt="0 0 0 xyz 0 0 0.53",
    mesh=((0, 0, 0), (0, 0, 0.24)),
    markers=[top_head_marker_head],
)



# The arm segment
shoulder_marker = Marker(
    name="SHOULDER", 
    parent_name="UPPER_ARM",
    position=(0, 0, 0),
)
upper_arm = Segment(
    name="UPPER_ARM",
    parent_name=trunk.name, 
    rt="0 0 0 xyz 0 0 0.53", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.28)),
    markers=[shoulder_marker],
)

elbow_marker = Marker(
    name="ELBOW",
    parent_name="LOWER_ARM",
    position=(0, 0, 0),
)
lower_arm = Segment(
    name="LOWER_ARM",
    parent_name=upper_arm.name,
    rt="0 0 0 xyz 0 0 -0.28",
    mesh=((0, 0, 0), (0, 0, -0.27)),
    markers=[elbow_marker],
)

wrist_marker = Marker(
    name="WRIST",
    parent_name="HAND",
    position=(0, 0, 0),
)
finger_marker = Marker(
    name="FINGER",
    parent_name="HAND",
    position=(0, 0, -0.19),
)
hand = Segment(
    name="HAND",
    parent_name=lower_arm.name,
    rt="0 0 0 xyz 0 0 -0.27",
    mesh=((0, 0, 0), (0, 0, -0.19)),
    markers=[wrist_marker, finger_marker]
)



# The thigh segment
thigh = Segment(
    name="THIGH", 
    parent_name=trunk.name, 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.42)),
)


# The shank segment
knee_marker = Marker(
    name="KNEE", 
    parent_name="SHANK",
    position=(0, 0, 0),
)
shank = Segment(
    name="SHANK", 
    parent_name=thigh.name, 
    rt="0 0 0 xyz 0 0 -0.42", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, -0.43)),
    markers=[knee_marker],
)


# The foot segment
ankle_marker = Marker(
    name="ANKLE", 
    parent_name="FOOT",
    position=(0, 0, 0),
)
toe_marker = Marker(
    name="TOE", 
    parent_name="FOOT",
    position=(0, 0, 0.25),
)
foot = Segment(
    name="FOOT", 
    parent_name=shank.name, 
    rt="0 0 0 xyz 0 0 -0.43", 
    rotations="x", 
    mesh=((0, 0, 0), (0, 0, 0.25)),
    markers=[ankle_marker, toe_marker],
)

# Put the model together, print it and print it to a bioMod file
kinematic_chain = KinematicChain(segments=(trunk, head, upper_arm, lower_arm, hand, thigh, shank, foot))
print(kinematic_chain)
kinematic_chain.write(kinematic_model_file_path)

In [ ]:
# Run this cell to show the model you just created. 
# Please note that due to some jupyter limitations, you may have to restart after running this cell
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(kinematic_model_file_path)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.set_q((0, 0, -0.15, 1.20, 0.7, -1, -1.11))

# Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
viz.exec()

# If nothing happens, check for background. Sometimes the window loads behind the current window

## Dynamic model
For that second part, we will add the inertia parameters to the model. 
It will allow us, for instance, to perform inverse dynamics analyses.

We are using the De Leva inertia parameter model.

In [ ]:
# So first let's focus on the extra three parameters we ignore before, that is the mass, the center_of_mass and the inertia_xxyyzz.
    
# Here is an example on how to use the class including these inertia parameters
my_new_segment = Segment(
    name="my_new_semgent_name", 
    parent_name="the_name_of_the_parent_segment", 
    rt="0 0 0 xyz 0 0 0",  # The transformation between the segment and its parent. The first three values are the three rotations about the given sequence (here xyz) and then the three translations
    translations="yxz",  # The degrees of fredom in translation
    rotations="xyz",  # The degrees of fredom in rotation
    mass=1,
    center_of_mass=(0, 0, 0.15),
    inertia_xxyyzz=(0.0001, 0.0001, 0.001),
    mesh=((0, 0, 0), (0, 0, 0.3))  # The list of mesh point to show to create the stick figure. All the points are link in order
)
print(my_new_segment)  # Later this print should be sent to a file

In [ ]:
# Now let's use the De Leva table to produce realistic inertia value for our model
import biorbd
import numpy

# Let's first copy the De Leva table in a dictionary using a simple DeLeva class
class DeLeva:
    # This is an internal class just to make things well sorted
    class Param:    
        def __init__(
            self,
            marker_names,  # The name of the markers medial/lateral
            mass,  # Percentage of the total body mass
            center_of_mass,  # Position of the center of mass as a percentage of the distance from medial to distal
            radii,  # [Sagittal, Transverse, Longitudinal] radii of giration
        ):
            self.marker_names = marker_names
            self.mass = mass
            self.center_of_mass = center_of_mass
            self.radii = radii

    def __init__(self, sex, mass, model):
        self.sex = sex  # The sex of the subject
        self.mass = mass  # The mass of the subject
        self.model = model  # The biorbd model. This is to compute lengths
        
        # Produce some easy to access variables
        self.q_zero = numpy.zeros((model.nbQ()))
        self.marker_names = [name.to_string() for name in model.markerNames()]
        
        # This is the actual copy of the De Leva table
        self.table = {}
        self.table["male"] = {}
        self.table["male"]["HEAD"]      = DeLeva.Param(marker_names=("TOP_HEAD", "SHOULDER"), mass=0.0694  , center_of_mass=0.5002, radii=(0.303, 0.315, 0.261))
        self.table["male"]["TRUNK"]     = DeLeva.Param(marker_names=("SHOULDER", "PELVIS")  , mass=0.4346  , center_of_mass=0.5138, radii=(0.328, 0.306, 0.169))
        self.table["male"]["UPPER_ARM"] = DeLeva.Param(marker_names=("SHOULDER", "ELBOW")   , mass=0.2710*2, center_of_mass=0.5772, radii=(0.285, 0.269, 0.158))
        self.table["male"]["LOWER_ARM"] = DeLeva.Param(marker_names=("ELBOW", "WRIST")      , mass=0.1620*2, center_of_mass=0.4574, radii=(0.276, 0.265, 0.121))
        self.table["male"]["HAND"]      = DeLeva.Param(marker_names=("WRIST", "FINGER")     , mass=0.0061*2, center_of_mass=0.7900, radii=(0.628, 0.513, 0.401))
        self.table["male"]["THIGH"]     = DeLeva.Param(marker_names=("PELVIS", "KNEE")      , mass=0.1416*2, center_of_mass=0.4095, radii=(0.329, 0.329, 0.149))
        self.table["male"]["SHANK"]     = DeLeva.Param(marker_names=("KNEE", "ANKLE")       , mass=0.0433*2, center_of_mass=0.4459, radii=(0.255, 0.249, 0.103))
        self.table["male"]["FOOT"]      = DeLeva.Param(marker_names=("ANKLE", "TOE")        , mass=0.0137*2, center_of_mass=0.4415, radii=(0.257, 0.245, 0.124))

        self.table["female"] = {}
        self.table["female"]["HEAD"]      = DeLeva.Param(marker_names=("TOP_HEAD", "SHOULDER"), mass=0.0668  , center_of_mass=0.4841, radii=(0.271, 0.295, 0.261))
        self.table["female"]["TRUNK"]     = DeLeva.Param(marker_names=("SHOULDER", "PELVIS")  , mass=0.4257  , center_of_mass=0.4964, radii=(0.307, 0.292, 0.147))
        self.table["female"]["UPPER_ARM"] = DeLeva.Param(marker_names=("SHOULDER", "ELBOW")   , mass=0.2550*2, center_of_mass=0.5754, radii=(0.278, 0.260, 0.148))
        self.table["female"]["LOWER_ARM"] = DeLeva.Param(marker_names=("ELBOW", "WRIST")      , mass=0.0138*2, center_of_mass=0.4559, radii=(0.261, 0.257, 0.094))
        self.table["female"]["HAND"]      = DeLeva.Param(marker_names=("WRIST", "FINGER")     , mass=0.0056*2, center_of_mass=0.7474, radii=(0.531, 0.454, 0.335))
        self.table["female"]["THIGH"]     = DeLeva.Param(marker_names=("PELVIS", "KNEE")      , mass=0.1478*2, center_of_mass=0.3612, radii=(0.369, 0.364, 0.162))
        self.table["female"]["SHANK"]     = DeLeva.Param(marker_names=("KNEE", "ANKLE")       , mass=0.0481*2, center_of_mass=0.4416, radii=(0.271, 0.267, 0.093))
        self.table["female"]["FOOT"]      = DeLeva.Param(marker_names=("ANKLE", "TOE")        , mass=0.0129*2, center_of_mass=0.4014, radii=(0.299, 0.279, 0.124))

    def segment_mass(self, segment):
        return self.table[self.sex][segment].mass * self.mass
    
    def segment_length(self, segment):
        table = self.table[self.sex][segment]
        
        # Find the position of the markers when the model is in resting position
        marker_positions = numpy.array([marker.to_array() for marker in self.model.markers(self.q_zero)]).transpose()
        
        # Find the index of the markers required to compute the length of the segment
        idx_proximal = self.marker_names.index(table.marker_names[0])
        idx_distal = self.marker_names.index(table.marker_names[1])
        
        # Compute the Euclidian distance between the two positions
        return numpy.linalg.norm(marker_positions[:, idx_distal] - marker_positions[:, idx_proximal])
        
    def segment_center_of_mass(self, segment, inverse_proximal=False):
        # This method will compute the length of the required segment based on the biorbd model and required markers
        # If inverse_proximal is set to True, then the value is returned from the distal position
        table = self.table[self.sex][segment]
        
        # Find the position of the markers when the model is in resting position
        marker_positions = numpy.array([marker.to_array() for marker in self.model.markers(self.q_zero)]).transpose()
        
        # Find the index of the markers required to compute the length of the segment
        idx_proximal = self.marker_names.index(table.marker_names[0])
        idx_distal = self.marker_names.index(table.marker_names[1])
        
        # Compute the position of the center of mass
        if inverse_proximal:
            center_of_mass = (1-table.center_of_mass) * (marker_positions[:, idx_proximal] - marker_positions[:, idx_distal])
        else:
            center_of_mass = table.center_of_mass * (marker_positions[:, idx_distal] - marker_positions[:, idx_proximal])
        return tuple(center_of_mass)  # convert the result to a Tuple which is good practise
        
    def segment_moment_of_inertia(self, segment):
        mass = self.segment_mass(segment)
        length = self.segment_length(segment)
        radii = self.table[self.sex][segment].radii
        
        return (mass * (length*radii[0])**2, mass * (length*radii[1])**2, mass * (length*radii[2])**2)

In [ ]:
# Let's suppose we have a male subject of 70kg which dimension are given by the bioMod previously created
deleva = DeLeva(sex="male", mass=70, model=biorbd.Model(kinematic_model_file_path))

# We can now easily compute the mass, center_of_mass and radii of giration for any segment
print(f"The mass of the TRUNK is: {deleva.segment_mass('TRUNK')}")
print(f"The length of the TRUNK is: {deleva.segment_length('TRUNK')}")
print(f"The center of mass of the TRUNK is situated at {deleva.segment_center_of_mass('TRUNK', inverse_proximal=True)} from the PELVIS")
print(f"The moment of inertia of the TRUNK is: {deleva.segment_moment_of_inertia('TRUNK')}")

In [ ]:
# So let's fill all the field in each segments (and again let's define a convenient function that will do that for us)
def fill_segment(segment, deleva, inverse_proximal=False):
    segment.mass = deleva.segment_mass(segment.name)
    segment.center_of_mass = deleva.segment_center_of_mass(segment.name, inverse_proximal)
    segment.inertia_xxyyzz = deleva.segment_moment_of_inertia(segment.name)


# Fill all the segments
fill_segment(trunk, deleva, True)
fill_segment(head, deleva, True)
fill_segment(upper_arm, deleva)
fill_segment(lower_arm, deleva)
fill_segment(hand, deleva)
fill_segment(thigh, deleva)
fill_segment(shank, deleva)
fill_segment(foot, deleva)

# Print and rewrite the bioMod file
print(kinematic_chain)
kinematic_chain.write(dynamic_model_file_path)

In [13]:
# Run this cell to show the model you just created. 
# You should notice that the centers of mass now appear
# Please note that due to some jupyter limitations, you may have to restart after running this cell
# Please also note that this cell will crash if runned from the binder

import bioviz

# Send the previously loaded model to the vizualizer
viz = bioviz.Viz(dynamic_model_file_path)

# Move the model to a recognizable position (arm raised and knee flexed)
viz.set_q((0, 0, -0.15, 1.20, 0.7, -1, -1.11))

# Halt the program so you can interact with the vizualiser. Closing the window should allow to continue
viz.exec()

# If nothing happens, check for background. Sometimes the window loads behind the current window